# The Competition

To familairize outselves with the data, and play around with CNNs, we all decided to make our own CNN architecture and see who could get the best results. 

In [ ]:
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
import os
from pandas import DataFrame
from sklearn.model_selection import train_test_split
DIRNAME = "data_generation/weather_images/"

## Ted's Model

**Why it will win:** I chose to use more epochs and Adam as opposed to RMSProp for my optimizer. I hope this will help the model converge more towards a global minimum in the cost function.

In [ ]:
def preprocess(directory):
  #The eventual list of all images
  
    images=[]
      #The eventual list of all labels where (0,1) is rainy and (1,0) is sunny
    labels=[]
    for root,dirs,files in os.walk(directory):
    for file in files:
    if file.endswith(".png"):
        fullpath=os.path.join(root,file)
        #Convert to a One-Hot-Encoding
        if "sunny" in file: 
            label=((1,0))
        else:
            label=((0,1))
        images.append(fullpath)
        labels.append(label)
    assert len(images)==len(labels)
    return images,labels

images, labels = preprocess(DIRNAME)

x_train,  x_test, y_train, y_test =  train_test_split(images, labels, test_size=0.25)

batch_size = 16
epochs = 15

In [ ]:
#Initialize the Model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.Adam()

# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['binary_accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


print('Not using data augmentation.')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)



# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)


# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

## Eric's Model

**Why it will win:** Using the ImageDataGeneration class adds more images with augmentations (I used featurewise_std_normalization, zca_whitening, rotation_range, width_shift_range, height_shift_range, horizontal_flip, rescale) to generate extra training data to learn from. It will also help the model generalize to unseen new data. This will give my model the advantage over the rest!


In [ ]:

tf.disable_eager_execution()

batch_size = 16
epochs = 10

DIR = DIRNAME

files = list(os.scandir(DIR))
X = [f.name for f in files]
y = ['sun' if f.name.startswith('sun') else 'rain' for f in files]

suns = sum((1 for w in y if w=='sun'))
print(suns,len(y)-suns)  

# The data, split between train and test sets:
name_train,  name_test, y_train, y_test =  train_test_split(X, y, test_size=0.33, random_state=42)

model = Sequential([
  Conv2D(32, (3, 3), padding='same',input_shape=(128,128,3)),
  Activation('relu'),
  Conv2D(32, (3, 3)),
  Activation('relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),

  Conv2D(64, (3, 3), padding='same'),
  Activation('relu'),
  Conv2D(64, (3, 3)),
  Activation('relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),

  Flatten(),
  Dense(512),
  Activation('relu'),
  Dropout(0.5),
  Dense(2),
  Activation('softmax')
])


# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop()

# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['binary_accuracy'])

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).

train_datagen = ImageDataGenerator(
    featurewise_std_normalization=True,
    zca_whitening=True
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale=1./255)

srcGen = train_datagen.flow_from_dataframe(
    DataFrame(data={'filename':name_train,'class':y_train}),directory=DIR,target_size=(128,128),batch_size=16)

dataGen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    DataFrame(data={'filename':name_test,'class':y_test}),
    directory=DIR,target_size=(128,128),batch_size=16)

# Fit the model on the batches generated by datagen.flow().
checkpointer = ModelCheckpoint(filepath='/data/Shared drives/Waymo Project/tmp/weights.hdf5', verbose=1, save_best_only=True)
model.fit_generator(srcGen,
                    epochs=epochs,
                    validation_data=dataGen,
                    workers=4,
                    callbacks=[checkpointer])


# Score trained model.
scores = model.evaluate(dataGen, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

## Jack's Model

**Why it will win:** I decided to use a larger architecture, with larger dense layers at the end and two times as many filters in each Convolution layer. This will let the model learn more complex patterns within the data at the expense of speed.


In [ ]:


images, labels = preprocess(DIRNAME)

x_train,  x_test, y_train, y_test =  train_test_split(images, labels, test_size=0.25)

batch_size = 16
epochs = 10

In [ ]:

model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['binary_accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


print('Not using data augmentation.')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)



# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4)


# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])